In [8]:
from examinebugs import examine_rev

distmodels = ["BasicDistanceModel", "GitDiffDistModel", 
              "MossDistModel", "PatientDiffDistModel"]
scoremodels = ["SimpleScoreModel", "LongestScoreModel", 
               "ShortestScoreModel", 
               "OutEdgesScoreModel", "SizeScoreModel", 
               "TimeWeightedScoreModel"]

In [18]:
buggy_dicts = {}
for dm in distmodels:
    for sm in scoremodels:
        name = "%s_%s" % (sm, dm)
        buggy_dicts[name] =  examine_rev(
            '/home/cat/rzou/linux', 'mm', 'mm', 
            dm, sm,
            '19be0eaffa3ac7d8eb6784ad9bdbc7d67ed8e619'
        )

Read patch models
Read patch models
Read patch models
Read patch models
Read patch models
Read patch models
Read patch models
Read patch models
Read patch models
Read patch models
Read patch models
Read patch models
Read patch models
Read patch models
Read patch models
Read patch models
Read patch models
Read patch models
Read patch models
Read patch models
Read patch models
Read patch models
Read patch models
Read patch models


In [3]:
for line in (eval(buggy_dict[674]['patch']['content'])):
    print line

static struct page *no_page_table(struct vm_area_struct *vma,
		unsigned int flags)
{
	/*
	 * When core dumping an enormous anonymous area that nobody
	 * has touched so far, we don't want to allocate unnecessary pages or
	 * page tables.  Return error instead of NULL to skip handle_mm_fault,
	 * then get_dump_page() will return NULL to leave a hole in the dump.
	 * But we can only make this optimization where a hole would surely
	 * be zero-filled if handle_mm_fault() actually did handle it.
	 */
	if ((flags & FOLL_DUMP) && (!vma->vm_ops || !vma->vm_ops->fault))
		return ERR_PTR(-EFAULT);
	return NULL;
}

static struct page *follow_page_pte(struct vm_area_struct *vma,
		unsigned long address, pmd_t *pmd, unsigned int flags)


In [21]:
buggy_dict.keys()
def change_name(name):
    return name \
        .replace("SizeScoreModel_BasicDistanceModel", "SizeScoreModel") \
        .replace("_", "\_") \
        .replace("Git", "Myer") \
        .replace("ScoreModel", "") \
        .replace("DistModel", "Dist") \
        .replace("BasicDistanceModel", "UnitDist") \
        .replace("TimeWeighted", "TimeWeightAvePath") \
        .replace("Shortest", "MinPath") \
        .replace("Longest", "MaxPath") \
        .replace("Size", "PatchLen") \
        .replace("Edges", "EdgesSum") \
        .replace("Simple", "AvePath")

In [31]:
for name, buggy_dict in buggy_dicts.iteritems():
    assert(buggy_dict.keys()[0] == 2984)
    assert(buggy_dict.keys()[1]==17944)
    percentiles = []
    for pid, p in buggy_dict.iteritems():
        percentiles.append(p['score'])
    percentiles.append(max(percentiles))
    percentiles.append(min(percentiles))
    percentiles.append(sum(percentiles)/len(percentiles))
    floats = map(lambda f: "%.4f" % f, percentiles)
    out = [change_name(name)]
    out.extend(floats)
    print " & ".join(out) + ' \\\\'


MaxPath\_MyerDiffDist & 461.0000 & 38444.0000 & 323.0000 & 323.0000 & 0.0000 & 38444.0000 & 0.0000 & 11142.0000 \\
AvePath\_MyerDiffDist & 461.0000 & 38444.0000 & 323.0000 & 323.0000 & 0.0000 & 38444.0000 & 0.0000 & 11142.1429 \\
AvePath\_MossDist & 0.3350 & 0.7850 & 0.2450 & 0.2450 & 0.0000 & 0.7850 & 0.0000 & 0.3421 \\
PatchLen\_MyerDiffDist & 57.0000 & 24.0000 & 4.0000 & 18.0000 & 650.0000 & 650.0000 & 4.0000 & 201.0000 \\
AvePath\_PatientDiffDist & 461.0000 & 35998.0000 & 323.0000 & 323.0000 & 0.0000 & 35998.0000 & 0.0000 & 10443.2857 \\
PatchLen & 57.0000 & 24.0000 & 4.0000 & 18.0000 & 650.0000 & 650.0000 & 4.0000 & 201.0000 \\
TimeWeightAvePath\_MossDist & 202608.0000 & 148176.0083 & 148176.0000 & 148176.0000 & 0.0000 & 202608.0000 & 0.0000 & 121392.0012 \\
OutEdgesSum\_MossDist & 0.2350 & 0.5400 & 0.2450 & 0.2450 & 0.0000 & 0.5400 & 0.0000 & 0.2579 \\
OutEdgesSum\_UnitDist & 1.0000 & 1.0000 & 1.0000 & 1.0000 & 0.0000 & 1.0000 & 0.0000 & 0.0000 \\
MinPath\_MyerDiffDist & 461.0000

In [29]:
for key, _ in buggy_dict.iteritems():
    for line in (eval(buggy_dict[key]['patch']['content'])):
        print line
    print '------------------------------------------'

}

static int faultin_page(struct task_struct *tsk, struct vm_area_struct *vma,
		unsigned long address, unsigned int *flags, int *nonblocking)
{
	struct mm_struct *mm = vma->vm_mm;
	unsigned int fault_flags = 0;
	int ret;

	/* For mlock, just skip the stack guard page. */
	if ((*flags & FOLL_MLOCK) &&
			(stack_guard_page_start(vma, address) ||
			 stack_guard_page_end(vma, address + PAGE_SIZE)))
		return -ENOENT;
	if (*flags & FOLL_WRITE)
		fault_flags |= FAULT_FLAG_WRITE;
	if (nonblocking)
		fault_flags |= FAULT_FLAG_ALLOW_RETRY;
	if (*flags & FOLL_NOWAIT)
		fault_flags |= FAULT_FLAG_ALLOW_RETRY | FAULT_FLAG_RETRY_NOWAIT;

	ret = handle_mm_fault(mm, vma, address, fault_flags);
	if (ret & VM_FAULT_ERROR) {
		if (ret & VM_FAULT_OOM)
			return -ENOMEM;
		if (ret & (VM_FAULT_HWPOISON | VM_FAULT_HWPOISON_LARGE))
			return *flags & FOLL_HWPOISON ? -EHWPOISON : -EFAULT;
		if (ret & VM_FAULT_SIGBUS)
			return -EFAULT;
		BUG();
	}

	if (tsk) {
		if (ret & VM_FAULT_MAJOR)
			tsk->maj_flt++;
	